This stores SQL Queries for our reference.

In [ ]:
-- 1. Drop existing tables to avoid conflicts and ensure correct vector dimensions
DROP TABLE IF EXISTS conversation_logs;
DROP TABLE IF EXISTS user_memories;

-- 2. Enable vector extension (just in case)
create extension if not exists vector;

-- 3. Create table for Semantic Logs (Gemini = 768 dimensions)
create table conversation_logs (
  id bigint generated by default as identity primary key,
  session_id text not null,
  role text not null,
  content text not null,
  embedding vector(768),  -- <--- CRITICAL: Must be 768 for Gemini
  created_at timestamp with time zone default timezone('utc'::text, now())
);

-- 4. Create table for Structured Facts
create table user_memories (
  id bigint generated by default as identity primary key,
  session_id text not null,
  memory_type text,
  content jsonb,
  confidence float,
  last_accessed timestamp,
  embedding vector(768)   -- <--- CRITICAL: Must be 768 for Gemini
);

-- 5. Create the search function
create or replace function match_conversation(
  query_embedding vector(768), 
  match_threshold float, 
  match_count int,
  p_session_id text
)
returns table (
  id bigint,
  content text,
  similarity float
)
language plpgsql
as $$
begin
  return query
  select
    conversation_logs.id,
    conversation_logs.content,
    1 - (conversation_logs.embedding <=> query_embedding) as similarity
  from conversation_logs
  where 1 - (conversation_logs.embedding <=> query_embedding) > match_threshold
  and session_id = p_session_id
  order by conversation_logs.embedding <=> query_embedding
  limit match_count;
end;
$$;

In [ ]:
create or replace function match_memories(
  query_embedding vector(768), 
  match_threshold float, 
  match_count int,
  p_session_id text
)
returns table (
  id bigint,
  memory_type text,
  content jsonb,
  similarity float
)
language plpgsql
as $$
begin
  return query
  select
    user_memories.id,
    user_memories.memory_type,
    user_memories.content,
    1 - (user_memories.embedding <=> query_embedding) as similarity
  from user_memories
  where 1 - (user_memories.embedding <=> query_embedding) > match_threshold
  and session_id = p_session_id
  order by user_memories.embedding <=> query_embedding
  limit match_count;
end;
$$;